In [1]:
import pandas as pd
import os, json
df_all = pd.read_csv(".data/available_TA_all.csv")
df_all = df_all[df_all["relevant"]==2]
df_topic = pd.read_csv("df_introduction.csv")


In [4]:
print(len(df_all))
print(len(df_all[~df_all["methods"].isna()]))

34070
33651


In [13]:
rows = df_all[~df_all["methods"].isna()].sample(2)
rows

,T_id,doi,title,abstract,authors,year,publisher,relevant,wos,scopus,pubmed,introduction,methods,results,discuession,conclustion
12640,TA022753,10.1016/j.fitote.2012.03.024,"Kujigamberol, a new dinorlabdane diterpenoid i...","A new compound, 15,20-dinor-5,7,9-labdatriene-...","Kimura, Ken-ichi; Minamikawa, Yuki; Ogasawara,...",2012,FITOTERAPIA,2,True,False,False,['1. introduction Amber is a fossilized tree r...,"['2. experimental ', '2.1. chemicals and strai...",To investigate the toxicity of M. alternifolia...,['3. results and discussion The major bioactiv...,NaN
31181,TA047380,10.1111/jcmm.15484,Nomilin targets the Keap1-Nrf2 signalling and ...,Osteoarthritis (OA) is a long-term and inflamm...,"Xue, Xing-He, Xue, Ji-Xin, Hu, Wei, Shi, Fang-...",2020,Journal of cellular and molecular medicine,2,False,False,True,Osteoarthritis (OA) is a prevalent senile diso...,Nomilin (purity > 98%) was purchased from Shan...,Figure 1A showed the chemical formula of NOM. ...,Osteoarthritis is one of the most widespread a...,NaN


In [20]:

prompt = """
The topic is {}
=================
Method:{}
Based on the above information:
 - First, summarize the methods and techniques used in the article, without any abbreviations or shorthand.
 - Then Divide the text reasonably, each paragraph should summarize which tasks the methods used in the text were applied to, what operations were performed on those substances, followed by the original text. Do not omit any information, but you can clean the text.
 - It is recommended to merge related paragraphs. Each output paragraph should be no less than 500 words.
 - No output in any format other than JSON is required. Translate it into English as a prompt.
EXAMPLE JSON OUTPUT:
{{
    "0001": {{
        "technology":[],
        "summery":"",
        "original_text":""，
    }},
    "0002": {{
        "technology":[],
        "summery":"",
        "original+text":"",
    }},
}}
The response is:
"""
llm_data = []
for _, row in df_all[~df_all["methods"].isna()].iterrows():
    T_id = row["T_id"]
    topic = df_topic[df_topic["T_id"]==T_id]["topic"].values[0]
    content = row["methods"]
    llm_input = prompt.format(topic, content)
    llm_data.append({
        "custom_id": T_id,
        "llm_input": llm_input
    })
len(llm_data)

33651

In [21]:
def data_to_jsonl_volcengine(datas:list, save_file_path=".tmp.jsonl"):
    import json
    # 火山引擎要求的jsonl格式
    with open(save_file_path, "w", encoding="utf-8") as f:
        for _data in datas:
            custom_id = _data["custom_id"]
            user_input = _data["llm_input"]
            _input = {
                "custom_id": custom_id,
                "body": {
                    "messages": [
                        {"role": "system","content": ""},
                        {"role": "user","content": user_input}
                    ],
                    "max_tokens": 10000,
                    "temperature": 0
                },
            }
            f.write(json.dumps(_input, ensure_ascii=False) + "\n")
    return

data_to_jsonl_volcengine(llm_data, "h_method_input.jsonl")

In [22]:
input_data = []
import json
with open("h_method_input.jsonl", "r") as r:
    for line in r:
        input_data.append(json.loads(line))

In [23]:
input_data[1523]

{'custom_id': 'TA001835',
 'body': {'messages': [{'role': 'system', 'content': ''},
   {'role': 'user',
    'content': '\nThe topic is Fractioning of orange (Citrus sinensis L.) essential oil using vacuum fractional distillation to concentrate minor compounds of industrial interest.\n=================\nMethod:[\'**2. materials and methods** \', \'2.1. obtainment of the orange essential oil Orange (*Citrus sinensis* L.) raw essential oil, harvested in 2015, was provided by company EcoCitrus, located in the city of Montenegro, Rio Grande do Sul, Brazil (the orchard of oranges from which the oil was obtained was also located at this location). The oil was obtained by cold pressing of the orange peels, being after stored in cold chamber (1 to 6 °C). The raw oil, before the fractionation process, was analyzed by GC/MS and HPLC to characterize it chemically. \', \'2.2. distillation system Accepted Manuscript The distillation system consisted in a glass column, packed with Raschig rings made 